In [1]:
import argparse
import os
import numpy as np
import math
import itertools
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
#os.makedirs("images10Healthy", exist_ok=True)

C:\Users\dmsss\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\dmsss\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\dmsss\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [17]:
import cv2

im = cv2.imread('data/Healthy/healthy/image_002.png')


print(im.shape)

(200, 200, 3)


In [52]:
img_shape = (3, 48,48)

In [53]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), 10))))
    z = sampled_z * std + mu
    return z


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.mu = nn.Linear(512, 10)
        self.logvar = nn.Linear(512, 10)

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        x = self.model(img_flat)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z

In [54]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img

In [55]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity


In [56]:
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()
cuda = True if torch.cuda.is_available() else False

if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()
dataloader = torch.utils.data.DataLoader(
    datasets.ImageFolder(
        './data/Healthy/',
        transform=transforms.Compose(
            [transforms.Resize(48), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=64,
    shuffle=True,
    drop_last=True,
)
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=0.0002, betas=(0.5,0.999)
)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5,0.999))
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


In [60]:
def sample_image(n_row, batches_done):
    z = Variable(Tensor(np.random.normal(0, 1, (n_row**2, 10))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "images10Healthy/%d.png" % batches_done, nrow=n_row, normalize=True)
for epoch in range(2000):
    for i, (imgs, _) in enumerate(dataloader):
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0))
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0))
        real_imgs = Variable(imgs.type(Tensor))
        optimizer_G.zero_grad()
        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)
        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )
        g_loss.backward()
        optimizer_G.step()
        optimizer_D.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 10))))
        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        d_loss.backward()
        optimizer_D.step()
        batches_done = epoch * len(dataloader) + i
        if batches_done % 200 == 0:
            sample_image(n_row=1, batches_done=batches_done)

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 512])

In [31]:
import cv2

im = cv2.imread('data/Healthy/healthy/image_002.png')
im2 = cv2.imread('data/Healthy/healthy/image_003.png')
im3=cv2.resize(im, (75,75))
im4=cv2.resize(im2, (75,75))
image1=[]
image1.append(im3)
image1.append(im4)
image1=np.array(image1)
im5 = cv2.imread('images10Healthy/crop.png')
im6 = cv2.imread('images10Healthy/crop2.png')
im5=cv2.resize(im5, (75,75))
im6=cv2.resize(im6, (75,75))
image2=[]
image2.append(im5)
image2.append(im6)
image2=np.array(image2)

print(image1.shape)
print(image2.shape)

(2, 75, 75, 3)
(2, 75, 75, 3)


In [34]:
import numpy as np
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image

def calculate_fid_score(images1, images2):
    # Load Inception-v3 model pre-trained on ImageNet
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(75,75, 3))

    # Preprocess and extract features for both sets of images
    def extract_features(images):
        images = preprocess_input(images)
        features = model.predict(images)
        return features

    features1 = extract_features(images1)
    features2 = extract_features(images2)

    # Calculate mean and covariance matrices
    mu1, sigma1 = np.mean(features1, axis=0), np.cov(features1, rowvar=False)
    mu2, sigma2 = np.mean(features2, axis=0), np.cov(features2, rowvar=False)

    # Calculate FID score
    fid_score = np.sum((mu1 - mu2)**2) + np.trace(sigma1 + sigma2 - 2 * sqrtm(sigma1 @ sigma2))

    return fid_score

# Example usage:
# images1 and images2 should be numpy arrays of shape (num_images, height, width, channels)
fid_score = calculate_fid_score(image1, image2)
print(f"FID Score: {fid_score}")


1/1 [==============================] - 0s 71ms/step
FID Score: (5316.4314830954545-1.3179245062129449e-05j)


In [10]:
print(sigma1)

1.4023704665624197


In [ ]:
# example of calculating the frechet inception distance in Keras for cifar10
# import numpy
# from numpy import cov
# from numpy import trace
# from numpy import iscomplexobj
# from numpy import asarray
# from numpy.random import shuffle
# from scipy.linalg import sqrtm
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.inception_v3 import preprocess_input
# from keras.datasets.mnist import load_data
# from skimage.transform import resize
# from keras.datasets import cifar10

# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	print(sigma1)
	print(sigma2)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov

	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(75,75,3))
# # load cifar10 images
# (images1, _), (images2, _) = cifar10.load_data()
# shuffle(images1)
# images1 = images1[:10000]
# print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (75,75,3))
images2 = scale_images(images2, (75,75,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

In [20]:
print(images1)

[[[[190. 168. 180.]
   [190. 168. 180.]
   [100.  85.  94.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  [[190. 168. 180.]
   [190. 168. 180.]
   [100.  85.  94.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  [[145. 126. 137.]
   [145. 126. 137.]
   [139. 123. 133.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  ...

  [[  5.   0.   5.]
   [  5.   0.   5.]
   [ 17.   1.   6.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  [[ 15.   1.   2.]
   [ 15.   1.   2.]
   [  7.   1.   2.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]

  [[ 15.   1.   2.]
   [ 15.   1.   2.]
   [  7.   1.   2.]
   ...
   [  0.   0.   0.]
   [  0.   0.   0.]
   [  0.   0.   0.]]]


 [[[ 49.  73.  43.]
   [ 49.  73.  43.]
   [111. 110.  77.]
   ...
   [109. 105.  77.]
   [108. 101.  77.]
   [108. 101.  77.]]

  [[ 49.  73.  43.]
   [ 49.  73.  43.]
   [111. 110.  77.]
   ...
   [109. 105.  77.]
 

In [23]:
print(images1.shape)

(10000, 75, 75, 3)


In [21]:
act1 = model.predict(images1)
mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)

313/313 [==============================] - 65s 165ms/step


In [22]:
print(sigma1)

[[ 131.29168978    7.30079487   11.86992794 ...  -10.53378983
  -176.94845262   51.99785779]
 [   7.30079487   99.36241764   -8.78737458 ...  -23.85319024
    -6.33794687   98.34252271]
 [  11.86992794   -8.78737458  716.02670726 ...  281.66404731
   -43.29398891   86.33783534]
 ...
 [ -10.53378983  -23.85319024  281.66404731 ... 5622.79008521
  1430.40261626   68.44627437]
 [-176.94845262   -6.33794687  -43.29398891 ... 1430.40261626
  4897.04570518 1316.0818881 ]
 [  51.99785779   98.34252271   86.33783534 ...   68.44627437
  1316.0818881  4298.8739953 ]]
